This notebook: 
- Loads project file from GitHub
- Loads assets from GitHub repo
- installs the custom language object 
- converts the training data to spaCy binary
- configure the project.yml file 
- train the model 
- assess performance 
- package the model (or push to huggingface) 


In [4]:
# temp to clear project folder
!rm -rf /srv/projects/course-materials/w2/using-inception-data/newlang_project


In [5]:
private_repo = False #@param {type:"boolean"}
repo_name = "repo-template" #@param {type:"string"}

!rm -rf /content/newlang_project
!rm -rf $repo_name
if private_repo:
    git_access_token = "" #@param {type:"string"}
    git_url = f"https://{git_access_token}@github.com/New-Languages-for-NLP/{repo_name}/"
    !git clone $git_url  -b main
    !cp -r ./$repo_name/newlang_project .  
    !mkdir newlang_project/assets/
    !mkdir newlang_project/configs/
    !mkdir newlang_project/corpus/
    !mkdir newlang_project/metrics/
    !mkdir newlang_project/packages/
    !mkdir newlang_project/training/
    !mkdir newlang_project/assets/$repo_name
    !cp -r ./$repo_name/* newlang_project/assets/$repo_name/
    !rm -rf ./$repo_name
else:
    !python -m spacy project clone newlang_project --repo https://github.com/New-Languages-for-NLP/$repo_name --branch main
    !python -m spacy project assets /srv/projects/course-materials/w2/using-inception-data/newlang_project

✔ Cloned 'newlang_project' from New-Languages-for-NLP/repo-template
/srv/projects/course-materials/w2/using-inception-data/newlang_project
✔ Your project is now ready!
To fetch the assets, run:
python -m spacy project assets /srv/projects/course-materials/w2/using-inception-data/newlang_project
ℹ Fetching 1 asset(s)
✔ Downloaded asset
/srv/projects/course-materials/w2/using-inception-data/newlang_project/assets/urban-giggle


In [6]:
# Install the custom language object from Cadet 
!python -m spacy project run install /srv/projects/course-materials/w2/using-inception-data/newlang_project


================================== install ==================================
Running command: rm -rf lang
Running command: mkdir lang
Running command: mkdir lang/yi
Running command: cp -r assets/urban-giggle/2_new_language_object/ lang/yi/yi
Running command: mv lang/yi/yi/setup.py lang/yi/
Running command: /srv/projects/course-materials/w2/venv/bin/python -m pip install -e lang/yi
Obtaining file:///srv/projects/course-materials/w2/using-inception-data/newlang_project/lang/yi
  Attempting uninstall: yi
    Found existing installation: yi 0.0.0
    Uninstalling yi-0.0.0:
      Successfully uninstalled yi-0.0.0
  Running setup.py develop for yi


In [2]:
# Create training config
!python -m spacy project run config /srv/projects/course-materials/w2/using-inception-data/newlang_project


=================================== config ===================================
Running command: /srv/projects/course-materials/w2/venv/bin/python -m spacy init config config.cfg --lang yi -F
⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: yi
- Pipeline: tagger, parser, ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy
Running command: /srv/projects/course-materials/w2/venv/bin/python scripts/update_config.py urban-giggle false


In [8]:
# Convert the conllu files from inception to spaCy binary format
# Currently requires edit to spacy/training/converters/conllu_to_docs.py line 194 
# if pos == "_":                                                                                                                  
#     pos = ""

!python -m spacy project run convert /srv/projects/course-materials/w2/using-inception-data/newlang_project -F


================================== convert ==================================
Running command: mkdir -p ./corpus/conllu
Running command: mkdir -p ./corpus/conll
Running command: mkdir -p ./corpus/converted
Running command: /srv/projects/course-materials/w2/venv/bin/python scripts/convert.py assets/urban-giggle/3_inception_export 10 yi
ℹ Grouping every 10 sentences into a document.
✔ Generated output file (108 documents):
corpus/converted/cu_proiel-ud-dev.spacy
ℹ Grouping every 10 sentences into a document.
✔ Generated output file (115 documents):
corpus/converted/cu_proiel-ud-test.spacy
ℹ Grouping every 10 sentences into a document.
✔ Generated output file (413 documents):
corpus/converted/cu_proiel-ud-train.spacy
ℹ Grouping every 10 sentences into a document.
Traceback (most recent call last):
  File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.8/runpy.py", line 87, in _run_code
    exec(code, 

In [3]:
# Read data files, convert to spaCy files
# test/train split 
!python -m spacy project run split /srv/projects/course-materials/w2/using-inception-data/newlang_project -F


=================================== split ===================================
Running command: /srv/projects/course-materials/w2/venv/bin/python scripts/split.py 0.2 11 yi
🚂 Created 1017 training docs
😊 Created 204 validation docs
🧪  Created 51 test docs


In [4]:
# Debug the data
!python -m spacy project run debug  /srv/projects/course-materials/w2/using-inception-data/newlang_project


=================================== debug ===================================
Running command: /srv/projects/course-materials/w2/venv/bin/python -m spacy debug data ./config.cfg

============================ Data file validation ============================
✔ Pipeline can be initialized with data
✔ Corpus is loadable

=============================== Training stats ===============================
Language: yi
Training pipeline: tok2vec, tagger, parser, ner
1017 training docs
204 evaluation docs
⚠ 166 training examples also in evaluation data
⚠ Low number of examples to train a new pipeline (1017)

============================== Vocab & Vectors ==============================
ℹ 92587 total word(s) in the data (9854 unique)
ℹ No word vectors present in the package

========================== Named Entity Recognition ==========================
ℹ 0 label(s)
0 missing value(s) (tokens with '-' label)
✔ Good amount of examples for all labels
✔ Examples without occurrences available for all la

In [5]:
# Train the model 
!python -m spacy project run train /srv/projects/course-materials/w2/using-inception-data/newlang_project


=================================== train ===================================
Running command: /srv/projects/course-materials/w2/venv/bin/python -m spacy train config.cfg --output training/urban-giggle --gpu-id -1 --nlp.lang=yi
ℹ Saving to output directory: training/urban-giggle
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2021-12-28 15:40:27,039] [INFO] Set up nlp object from config
[2021-12-28 15:40:27,047] [INFO] Pipeline: ['tok2vec', 'tagger', 'parser', 'ner']
[2021-12-28 15:40:27,051] [INFO] Created vocabulary
[2021-12-28 15:40:27,051] [INFO] Finished initializing nlp object
[2021-12-28 15:40:31,912] [INFO] Initialized pipeline components: ['tok2vec', 'tagger', 'parser', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'tagger', 'parser', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS TAGGER  LOSS PARSER  LOSS NER  TAG_ACC  DEP_UAS

In [ ]:
# Evaluate the model 
!python -m spacy project run evaluate /srv/projects/course-materials/w2/using-inception-data/newlang_project

In [1]:
# Package the model 
!mkdir ./export 
!python -m spacy package ./newlang_project/training/urban-giggle/model-last ./export 

/bin/bash: python: command not found


python: can't open file 'prodigy': [Errno 2] No such file or directory
